In [13]:
import os
import subprocess
import matplotlib.pyplot as plt
from scipy import signal, ndimage, stats
from scipy.io import wavfile
import numpy as np
import time
import pysubs2
import tensorflow as tf
import tensorflow_probability as tfp
from spatial_transformer_1d import SpatialTransformer1d

print(tf.test.is_gpu_available())
np.set_printoptions(precision=4)

True


In [14]:
# input_file = '/home/ltetrel/Naruto Shippuuden (2007)/Season 18/Naruto Shippuuden - S18E387.mkv'
input_file = '/home/ltetrel/Naruto Shippuuden (2007)/Season 01/Naruto Shippuuden - S01E001.mkv'

# should list all *.srt files there, so sub-resync is launched for each srt file
input_subfile = '/home/ltetrel/Naruto Shippuuden (2007)/Season 01/Naruto Shippuuden - S01E001.fr.srt'

output_subfile = input_subfile[:-4] + "_opt" + ".srt"
output_wav = " ".join(input_file.split(".")[:-1]) + ".wav"
vocal_file = output_wav[:-4] + "/vocals_{}.wav"

In [15]:
# extract the audio stream from movie
if not os.path.exists(output_wav):
    process = subprocess.Popen(["ffmpeg", "-y", "-i", input_file, output_wav]
                               , stdout=subprocess.PIPE
                               , stderr=subprocess.PIPE)
    print(process.stdout.read().decode("utf-8"))

In [16]:
# get audio duration
process = subprocess.Popen(["ffprobe"
                            , "-v"
                            , "error"
                            , "-show_entries"
                            , "format=duration"
                            , "-of"
                            , "default=noprint_wrappers=1:nokey=1"
                            , output_wav]
                           , stdout=subprocess.PIPE
                           , stderr=subprocess.PIPE)
duration_audio_in_ms = int(float(process.stdout.read().decode("utf-8"))*1000)

In [17]:
#extract the vocals from input .wav with spleeter, and load it

# Because spleeter uses a lot of RAM, we cannot extract the vocal from the whole file.
# Depending on the RAM space, we loop to get all spleeter vocals
max_duration_in_ms = 600000
vocal_offset = 0
nb_iter = int(duration_audio_in_ms/max_duration_in_ms)
vocal_resampled = np.array([], dtype=float)

for i in range(nb_iter + 1):
    if not os.path.exists(vocal_file.format(i)):
        process = subprocess.call(["spleeter"
                                   , "separate"
                                   , "--verbose"
                                   , "-i"
                                   , output_wav
                                   , "-p"
                                   , "spleeter:2stems"
                                   , "--offset"
                                   , str(vocal_offset)
                                   , "-d"
                                   , str(max_duration_in_ms/1000)
                                   , "-f"
                                   , "{filename}/{instrument}_" + str(i) +".wav"
                                   , "-o"
                                   , os.path.dirname(output_wav)]
                                   , stdout=subprocess.PIPE)
    vocal_offset = vocal_offset + int(max_duration_in_ms/1000)
    
    # reading vocal stem
    fs, vocal = wavfile.read(vocal_file.format(i))
    
    # sampling the signal (one sample each ms)
    sampling_rate = 1000
    n_samples = int(vocal.shape[0]*(sampling_rate/fs))
    resample_idx = np.insert((fs/sampling_rate)*np.ones(n_samples - 1), 0, 0.)
    resample_idx = np.array(np.around(np.cumsum(resample_idx)),dtype=int)
    vocal = vocal[resample_idx, 0]
    
    vocal_resampled = np.concatenate((vocal_resampled, vocal))  

In [18]:
# signal preprocessing
vocal_resampled = vocal_resampled**2

# We apply a manual threshold which correspond to an audio signal
threshold = np.quantile(vocal_resampled, 1/3)
vocal_predict = np.array(vocal_resampled > threshold, dtype=np.int32)

In [19]:
#create the box regressors (aka, the start and stop times)
subs = pysubs2.load(input_subfile, encoding="utf-8")

# rescaling sub or audio so they have the same length
duration_sub_in_ms = subs[-1].end
n_subs = len(subs)
if duration_sub_in_ms <= duration_audio_in_ms:
    duration_in_ms = duration_audio_in_ms
else:
    duration_in_ms = duration_sub_in_ms
    vocal_predict = np.concatenate((vocal_predict, np.zeros(duration_sub_in_ms - duration_audio_in_ms)))

# box signal correspond to the subtitles
boxes = np.zeros((duration_in_ms,), dtype=int)
starts = np.array([], dtype=int)
ends = np.array([], dtype=int)
for sub in subs:
    start = np.int32(sub.start)
    end = np.int32(sub.end)
    starts = np.append(starts, start)
    ends = np.append(ends, end)
    boxes[start:end] = 1

In [20]:
# preprocessing for smoothing value for back-propagation
def gaussian_kernel_1d(filter_length):
    #99% of the values
    sigma = (filter_length/2)/2.33
    width = int(filter_length/2)

    norm = 1.0 / (np.sqrt(2*np.pi) * sigma)
    kernel = [norm * np.exp((-1)*(x**2)/(2 * sigma**2)) for x in range(-width, width)]  

    return np.float32(kernel / np.sum(kernel))

kernel_size = 500
f = tf.reshape(gaussian_kernel_1d(kernel_size), [-1, 1, 1])
vocal_predict_filtered = tf.reshape(tf.constant(vocal_predict, dtype=tf.float32), [1, -1, 1])
vocal_predict_filtered = tf.reshape(tf.nn.conv1d(vocal_predict_filtered, filters=f, stride=1, padding='SAME'), [-1])

# kernel_size = kernel_size*4
# f = tf.reshape(gaussian_kernel_1d(kernel_size), [-1, 1, 1])
boxes_filtered = tf.reshape(tf.constant(boxes, dtype=tf.float32), [1, -1, 1])
boxes_filtered = tf.reshape(tf.nn.conv1d(boxes_filtered, filters=f, stride=1, padding='SAME'), [-1])

# clustering the subs into n groups
max_duration_allowed = 25000
mask_boxes_filtered = np.zeros_like(boxes_filtered)
num_sub = len(subs)
middle_subs = np.array([], dtype=np.int32)
for i in range(num_sub - 1):
    dura = np.int32(subs[i+1].start) - np.int32(subs[i].end)
    if(dura > max_duration_allowed):
        middle_sub = np.int32((np.int32(subs[i].end) + np.int32(subs[i+1].start))/2)
        middle_subs = np.append(middle_subs, [middle_sub])
mask_boxes_filtered[middle_subs] = 1
mask_boxes_filtered = np.cumsum(mask_boxes_filtered)

In [21]:
def loss(y, target):
    y = y[1:] - y[:-1]
    target = target[1:] - target[:-1]
    loss = tf.abs(1. - tfp.stats.correlation(x=y, y=target, event_axis=None))
    return loss

def regularizer(model):
    return tf.reduce_mean((model.B[1:] - model.B[:-1])**2)

@tf.function
def train(model, x, target, l=0.):
    with tf.GradientTape() as tape:
        loss_value = loss(model(x), target)
        if l > 0.:
            loss_value = loss_value + l*regularizer(model)
    return loss_value, tape.gradient(loss_value, model.training_vars)

# define inputs
x = boxes_filtered
target = vocal_predict_filtered #derivative remove one point

# learning parameters
lr_w = 1e-5
lr_b = 1e0
lr_B = lr_b
n_iters = 1000
size_x = tf.shape(x)[-1]
l = 0

model = SpatialTransformer1d()
model_non_rigid = SpatialTransformer1d(rigid=False, mask=mask_boxes_filtered, max_offset_range=500)

# opt = tf.keras.optimizers.SGD(learning_rate=1e-6)
opt_w = tf.keras.optimizers.Adam(learning_rate=lr_w)
opt_b = tf.keras.optimizers.Adam(learning_rate=lr_b)
opt_B = tf.keras.optimizers.Adam(learning_rate=lr_B)

In [22]:
start_time = time.time()

### training
### rough exploration

# offset and weight
init_params = [1., 0.]
range_min = [-1e-2, -5000]
range_max = [1e-2, 5000]

weights = [np.linspace(init_params[0] + range_min[0], init_params[0] + range_max[0], 20)
           , np.linspace(init_params[1] + range_min[1], init_params[1] + range_max[1], 20)]

cost = np.zeros((len(weights[0]), len(weights[1])), dtype=np.float32)

# resync
# for offset in [params[0]]:
for i in range(len(weights[0])):
    for j in range(len(weights[1])):
        # update model
        model.update_weights([weights[0][i]], [weights[1][j]])
        # compute cost (convolution of vocals with subtitles)
        cost[i, j] = loss(model(x), target)

argm = np.unravel_index(np.argmin(cost), cost.shape)
params = [weights[0][argm[0]], weights[1][argm[1]]]
print(params)
print(np.min(cost))
print("------")

# model.update_weights(W=[params[0]], b=[params[1]])
model_non_rigid.update_weights(W=[params[0]], b=[params[1]])
### gradient descent
for i in range(n_iters):
#     loss_value, grads = train(model, x, target)
#     if i % 50 == 0:
#         print("rigid - Step: {} - loss: {} - Params: W {}, b {} - Grad: w {} b: {}".format(i, loss_value.numpy(), model.W.numpy(), model.b.numpy(), grads[0], grads[1]))
    
#     opt_w.apply_gradients(zip(grads[:1], model.training_vars[:1]))
#     opt_b.apply_gradients(zip(grads[1:], model.training_vars[1:]))
#     losses_rigid = np.append(losses_rigid, loss_value)
    
    loss_value, grads = train(model_non_rigid, x, target, l)
    if i % 50 == 0:
        print("non-rigid - Step: {} - loss: {} - Params: W {}, b {}, B {}".format(i, loss_value.numpy(), model_non_rigid.W.numpy(), model_non_rigid.b.numpy(), np.mean(model_non_rigid.B.numpy())))
        print("\tGrads: W {}, b {}, B {}".format(np.mean(grads[0]), np.mean(grads[1]), np.mean(grads[2])))
        
    opt_w.apply_gradients(zip(grads[:1], model_non_rigid.training_vars[:1]))
    opt_b.apply_gradients(zip(grads[1:2], model_non_rigid.training_vars[1:2]))
    opt_B.apply_gradients(zip(grads[2:], model_non_rigid.training_vars[2:]))
    
print("%.2f s"%(time.time() - start_time))

[0.998421052631579, 263.1578947368416]
0.8610201
------
non-rigid - Step: 0 - loss: 0.9264902472496033 - Params: W [0.9984], b [263.1579], B 4.613520954155348e-10
	Grads: W 77.3369140625, b -4.951514711137861e-05, B -8.855233318172395e-06
non-rigid - Step: 50 - loss: 0.9183753728866577 - Params: W [0.9984], b [305.3376], B -9.498015403747559
	Grads: W 10.034515380859375, b -2.7455695089884102e-05, B -1.5719397197244689e-06
non-rigid - Step: 100 - loss: 0.9164296388626099 - Params: W [0.9983], b [323.7947], B -21.40962028503418
	Grads: W -8.620330810546875, b -1.4183096936903894e-05, B -3.909269253199454e-06
non-rigid - Step: 150 - loss: 0.9151971936225891 - Params: W [0.9983], b [327.8819], B -34.62298583984375
	Grads: W -5.166412353515625, b -1.3542114174924791e-05, B -1.8650978290679632e-06
non-rigid - Step: 200 - loss: 0.913986086845398 - Params: W [0.9983], b [329.4911], B -48.22711944580078
	Grads: W 14.686935424804688, b 1.7472892068326473e-05, B -3.2211869438469876e-07
non-rigid

In [ ]:
# saving subtitles for rigid
# optimal parameters
params = [1/model.training_vars[0].numpy(), (-1)*model.training_vars[1].numpy()]
print(params)

opt_starts = starts*params[0] + params[1]
opt_ends = ends*params[0] + params[1]
for sub, i in zip(subs, range(len(subs))):
    sub.start =  opt_starts[i]
    sub.end = opt_ends[i]
subs.save(output_subfile)

In [ ]:
# saving subtitles for non-rigid
# optimal parameters
params = [1/model_non_rigid.training_vars[0].numpy(), (-1)*model_non_rigid.training_vars[1].numpy(), (-1)*model_non_rigid.training_vars[2].numpy()]
print(params)

for sub, i in zip(subs, range(len(subs))):
    id_mask = int(sub.start)
    sub.start = int(params[0]*sub.start + params[1] + params[2][int(sub.start)])
    sub.end = int(params[0]*sub.end + params[1] + params[2][int(sub.end)])
subs.save(output_subfile + ".non_rigid.srt")

In [12]:
# saving subtitles for masked non-rigid
# optimal parameters
params = [1/model_non_rigid.training_vars[0].numpy(), (-1)*model_non_rigid.training_vars[1].numpy(), (-1)*model_non_rigid.training_vars[2].numpy()]
print(params)

for sub, i in zip(subs, range(len(subs))):
    id_mask = int(mask_boxes_filtered[int(sub.start)])
    sub.start = int(params[0]*sub.start + params[1] + params[2][id_mask])
    sub.end = int(params[0]*sub.end + params[1] + params[2][id_mask])
subs.save(output_subfile + ".non_rigid.srt")

[array([1.0018], dtype=float32), array([-377.351], dtype=float32), array([ -18.9268, -165.5825], dtype=float32)]


In [ ]:
subs_opt = pysubs2.load(output_subfile, encoding="utf-8")

boxes_opt = np.zeros((duration_in_ms,), dtype=np.int32)
starts_opt = np.array([], dtype=np.int32)
ends_opt = np.array([], dtype=np.int32)
for sub in subs_opt:
    start = np.int32(sub.start)
    end = np.int32(sub.end)
    starts_opt = np.append(starts_opt, start)
    ends_opt = np.append(ends_opt, end)
    boxes_opt[start:end] = 1

In [ ]:
%matplotlib
plt.subplot(2,1,1)
plt.plot(model_non_rigid(x) - target)
plt.title("non-rigid")
plt.subplot(2,1,2)
plt.plot(model(x) - target)
plt.title("rigid")
plt.show()

In [ ]:
%matplotlib
plt.plot(target, "k")
plt.plot(model_non_rigid(x), "g--")
plt.plot(model(x), "r--")
plt.show()

In [ ]:
%matplotlib
# cost function over params
plt.figure()
plt.imshow(cost[::-1, :], extent=[init_params[1] + range_min[1]
                         , init_params[1] + range_max[1]
                         , init_params[0] + range_min[0]
                         , init_params[0] + range_max[0]], aspect="auto", cmap="gray")
plt.xlabel("offset")
plt.ylabel("ratio")

In [ ]:
%matplotlib
# time = np.arange(data.size)/(1000)
fig, ax = plt.subplots()
# plt.plot(time, data)
# plt.plot(time[:vocals.size], vocals)
# plt.plot(time, boxes)
# plt.plot(vocal_predict)
# plt.plot(boxes, "r")
# plt.plot(boxes_new, "g")
plt.plot(vocal_predict_filtered, "k")
plt.plot(boxes_filtered, "g--")
plt.plot(mask_boxes_filtered / np.max(mask_boxes_filtered), "g")
# plt.plot(idx_boxes, "g")
plt.xlabel("time (ms)")
plt.show()